# Main Model for Handwritten Text Synthesis GAN

# This file is for demonstration purposes only
This model will consist of 4 major networks, following the general architecture of an GAN.

1. Encoder: Produces an embedding that will be concatenated with the noise vector.
2. Generator: Taking noise vector as input and the text embedding to produce an 128x2048 image.
3. Discriminator: Trained alternating with generator input and ground-truth input, binary classification real or fake.
4. Recognizer: Taking image as input, produce a vector representation of the text.

## Imports

In [ ]:
import csv
import datetime
import os
import random
import re

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import torch
import torch_fidelity
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
from torch.nn.utils.spectral_norm import spectral_norm
from torch.utils.data import DataLoader, Dataset, RandomSampler, Subset, random_split
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.text import CharErrorRate
from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import Grayscale, Resize, ToPILImage, ToTensor
from torchvision.transforms.functional import InterpolationMode
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

## Data Loading

### Helper Functions (Run once only to format data)

In [ ]:
SCALE_HEIGHT = 32
SCALE_WIDTH = SCALE_HEIGHT*16

def preprocess_lines(data_root):
    """
    Creates a new `.txt` file `lines_improved.txt` that will be used
    for querying. This new `.txt` file contains all info necessary
    for the functionality of this project.
    """

    original_path = os.path.join(data_root, "lines.txt")
    improved_path = os.path.join(data_root, "lines_improved.txt")
    fi = open(improved_path, "w")

    # Some variables for tracking
    num_samples = 0
    valid_samples = 0

    # Loop through "lines.txt"
    with open(original_path, "r") as fo:
        headers = ["image_id", "image_path", "image_pt_path", "graylevel", "original_height", "original_width", "transcription", "transcription_len"]

        # First write the headers at the top of the file
        fi.writelines("\t".join(headers) + "\n")

        # Skip the intro stuff
        for line in fo.readlines():
            if line.startswith("#"):
                continue

            # Valid lines, not the intro_text
            line_items = line.strip().split(" ")  # `strip()` to remove newlines

            # The actual items (we extract the important ones)
            image_id = line_items[0]
            status = line_items[1]
            graylevel = int(line_items[2])
            transcription = " ".join(line_items[8:])  # Some data has whitespace, we join string till the end

            # Skip error images
            if status == "err":
                continue

            # Alphanumeric + common punctuation regex
            # Returns None if no match
            # 26 + 26 + 10 + 9 + 1 = 72
            # Spaces might be included as well
            # Punctuation include , ! ? ' " , : ; -
            if re.fullmatch("[a-zA-Z0-9.!?'\",:;| -]*", transcription) is None:
                continue

            # Now we have valid transcription
            num_samples += 1

            # We get the `.png` image path
            inp = image_id.split("-")  # `inp` stands for image name parts
            image_path_head = os.path.join(data_root, "lines", inp[0], f"{inp[0]}-{inp[1]}")
            image_path_tail = f"{image_id}.png"
            image_path = os.path.join(image_path_head, image_path_tail)

            # Read image, gets its dimensions, perform processing operations, and other stuff
            tmp_image = cv.imread(os.path.join(image_path_head, image_path_tail), cv.IMREAD_GRAYSCALE)  # Removes the channel dimension
            height, width = tmp_image.shape

            # Scaling calculations
            # If width * scale >= desired length (>= to be safe)
            # Condition here to speed up overall processing time
            if width * (SCALE_HEIGHT/height) >= SCALE_WIDTH:
                continue

            resized_tensor = process_image(tmp_image, graylevel)
            image_pt_path = os.path.join(image_path_head, f"{image_id}.pt")
            torch.save(resized_tensor, image_pt_path)

            # A fully valid image
            # Separate by underscores because `transcription` has spaces so we can't split by spaces
            fi.writelines(f"{image_id}\t{image_path}\t{image_pt_path}\t{graylevel}\t{height}\t{width}\t{transcription}\t{len(transcription)}\n")
            valid_samples += 1

        fi.close()

    print("# samples:", num_samples)
    print("Valid samples:", valid_samples)


def process_image(cv_image, graylevel):
    """
    Takes in a grayscale image that OpenCV read of shape (H, W) of type uint8
    Returns a PyTorch tensor of shape (1, 32, W'), where W' is the scaled width
    This tensor is padded and effectively thresholded
    """

    # Scaling factor
    height, width = cv_image.shape
    scale = SCALE_HEIGHT/height
    scaled_width = int(width*scale)

    # Trick here is to apply threshold before resize and padding
    # This allows OpenCV resizing to create a cleaner output image
    # 2nd return value is the thresholded image
    output = cv.threshold(cv_image, graylevel, 255, cv.THRESH_BINARY)[1]

    # INTER_AREA recommended for sizing down
    output = cv.resize(output, (scaled_width, SCALE_HEIGHT), interpolation=cv.INTER_AREA)

    # Turn it back to a tensor and map to [0, 1]
    output = torch.from_numpy(output).unsqueeze(0).type(torch.float32)
    output = (output-output.min()) / (output.max()-output.min())

    # Add padding
    _, _, resized_height = output.shape
    padding_to_add = SCALE_WIDTH - resized_height
    output = F.pad(output, (0, padding_to_add), value=1.0)

    return output

# Uncomment this if your data isn't processed yet
preprocess_lines("../data/")

### Integer ⇔ Char Mapping Dict (Run everytime before training)

In [ ]:
# Sorted by ascii code
valid = [
    ' ', '!', '"', "'", ',', '-', '.',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    ':', ';', '?',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'
]
# Enumerate from 1 to save space for padding
# Reserve 0 for CTC blank
char_to_int = {v: i for i, v in enumerate(valid, 1)}
int_to_char = {i: v for i, v in enumerate(valid, 1)}

### Defining Dataset Class

In [ ]:
class LineDataset(Dataset):
    def __init__(self, lines_improved_dir, ty=None):
        """
        params:
            lines_improved_dir: path to the `lines_improved.txt` file
            ty: type of the dataset "txt", "img" for text dataset or image dataset.
        """

        # Dataframe containing the stuff in `lines_improved.txt`
        self.lines_df = pd.read_csv(lines_improved_dir, sep="\t", quoting=csv.QUOTE_NONE)

        # Class properties
        self.ty = ty  # Type of dataset (lines, images, or both)
        self.max_transcription_len = max(self.lines_df["transcription_len"])

        # Temp variables...
        length = self.lines_df.shape[0]
        line_datas = self.lines_df.iloc
        ret_texts = [line_datas[i]["transcription"].replace('|', ' ') for i in range(length)]
        ret_ctois = [torch.tensor([char_to_int[char] for char in ret_texts[i]]) for i in range(length)]

        # ...for the important data
        if self.ty in ("txt", None):  # Added this condition to speed thigns up if only text
            self.ret_ctoi_paddeds = [F.pad(ret_ctois[i], pad=(0, self.max_transcription_len-len(ret_ctois[i]))) for i in range(length)]
        if self.ty in ("img", None):
            self.ret_images = [torch.load(line_datas[i]["image_pt_path"]) for i in range(length)]

    def __len__(self):
        return len(self.lines_df)

    def __getitem__(self, index):
        # Different type of individual loaders
        if self.ty == "txt":
            return self.ret_ctoi_paddeds[index]
        elif self.ty == "img":
            return self.ret_images[index]
        else:
            return self.ret_images[index], self.ret_ctoi_paddeds[index]

### Initializing Datasets and Subsets of Datasets

In [ ]:
line_transcription_dataset = LineDataset("../data/lines_improved.txt", ty="txt")
line_image_dataset = LineDataset("../data/lines_improved.txt", ty="img")
line_dataset = LineDataset("../data/lines_improved.txt")

# Don't change this, we want to maintain consistent split
torch.manual_seed(12345678)  # DO NOT REMOVE THIS LINE
line_transcription_dataset_train, line_transcription_dataset_val = random_split(line_transcription_dataset, [0.8, 0.2])
line_image_dataset_train, line_image_dataset_val = random_split(line_image_dataset, [0.8, 0.2])
line_dataset_train, line_dataset_val = random_split(line_dataset, [0.8, 0.2])

# To train on a small dataset
line_transcription_dataset_train = Subset(line_transcription_dataset_train, range(100))
line_transcription_dataset_val = Subset(line_transcription_dataset_val, range(20))

line_image_dataset_train = Subset(line_image_dataset_train, range(100))
line_image_dataset_val = Subset(line_image_dataset_val, range(20))

line_dataset_train = Subset(line_dataset_train, range(2500))
line_dataset_val = Subset(line_dataset_val, range(500))

# line_transcription_dataset_train, line_transcription_dataset_val, _ = random_split(line_transcription_dataset, [0.005, 0.005, 0.99])
# line_image_dataset_train, line_image_dataset_val, _ = random_split(line_image_dataset, [0.005, 0.005, 0.99])
# line_dataset_train, line_dataset_val = random_split(line_dataset, [0.0025, 0.9975])

print("lines")
print(len(line_transcription_dataset_train), len(line_transcription_dataset_val))
print("images")
print(len(line_image_dataset_train), len(line_image_dataset_val))
print("both")
print(len(line_dataset_train), len(line_dataset_val))


### Showing Example Image and Label

In [ ]:
image, label = line_dataset_train[0]
print(image.shape)
plt.title("".join([int_to_char[int(val)] for val in label[label.nonzero()]]))
print(image.squeeze(0).shape)
plt.imshow(image.squeeze(0), cmap='gray')
label, "".join([int_to_char[int(val)] for val in label[label.nonzero()]])
# line_dataset.lines_df.iloc[798]
print(image)

## Define Key Functions

### Recognizer Related Functions

In [ ]:
def calculate_recog_accuracy(preds, target):
    """
    Calculates the accuracy of the recognizer with character error rate
    which is based on edit distance

    Params:
        preds: a list of prediction strings
        targets: a list of target strings

    Returns:
        An integer, the character error rate average across
        all predictions and targets
    """

    cer = CharErrorRate()
    return cer(preds, target)


def create_strings_from_tensor(int_tensor):
    """
    Params:
        int_tensor: A shape (N, 82) tensor where each row corresponds to
        a integer mapping of a string. Includes padding

    Returns:
        A list of N strings
    """

    strings = []
    for string_map in int_tensor:
        strings.append("".join([int_to_char[int(i)] for i in string_map[string_map != 0]]))
    return strings

### Plotting and Evaluating Functions

In [ ]:
"""Dataloading Functions"""
"""Datasaving Functions"""

"""Plotting Functions"""
"""Evaluation Functions"""
def calculate_gan_loss_and_accuracies(generator, encoder, discriminator, recognizer,
                                  line_loader, fid_set,
                                  batch_size=64, adversarial_loss_function=nn.BCELoss(), recognizer_loss_function=nn.NLLLoss(),
                                  device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'), recognizer_downscale_factor=1):
    """
    Calculates the loss for the GAN

    Params:
        generator: The generator model
        encoder: The encoder model
        discriminator: The discriminator model
        recognizer: The recognizer model
        line_loader: The dataloader for lines and their corresponding texts
        batch_size: The batch size to use
        adversarial_loss_function: The adversarial loss function to use
        recognizer_loss_function: The recognizer loss function to use

    Returns:
        generator_and_encoder_loss: The loss for the generator and encoder
        discriminator_loss: The loss for the discriminator
    """
    fid_set.reset()  # Just to make sure
    generator_and_encoder_loss = 0
    discriminator_loss = 0
    discriminator_accuracy = 0
    recognizer_error = 0  # Character error rate
    with torch.no_grad():
        for i, (line_image_batch, input_text_batch) in enumerate(line_loader):
            line_image_batch = line_image_batch.to(device)
            input_text_batch = input_text_batch.to(device)

            # generate noise of N x noise_dim
            noise = torch.randn(len(input_text_batch), generator.noise_dim).to(device)

            # Use encoder and generator to generate fake images
            text_embedding = encoder(input_text_batch)
            fake_image_batch = generator(noise, text_embedding)

            # train discriminator
            discriminator_output_for_real_images = discriminator(line_image_batch)
            discriminator_output_for_fake_images = discriminator(fake_image_batch)
            label_for_real_images = torch.ones_like(discriminator_output_for_real_images).to(device)
            label_for_fake_images = torch.zeros_like(discriminator_output_for_fake_images).to(device)
            real_images_loss = adversarial_loss_function(discriminator_output_for_real_images, label_for_real_images)
            fake_images_loss = adversarial_loss_function(discriminator_output_for_fake_images, label_for_fake_images)
            discriminator_loss_eval = (real_images_loss + fake_images_loss) / 2
            discriminator_loss += discriminator_loss_eval.item()
            discriminator_accuracy += (torch.sum(discriminator_output_for_real_images >= 0.5).item() + torch.sum(discriminator_output_for_fake_images < 0.5).item()) / (2 * batch_size)

            # train generator

            adversarial_loss = adversarial_loss_function(discriminator_output_for_fake_images, label_for_real_images)
            recognizer_outputs = recognizer(fake_image_batch)
            recognizer_loss = recognizer_loss_function(
                # Takes input torch.Size([N, 73, 82])
                F.log_softmax(recognizer_outputs, 1),
                input_text_batch  # Shape N by 82
            )

            # Recognizer loss in general bigger than recognizer+encoder loss
            recognizer_loss = recognizer_loss / recognizer_downscale_factor

            generator_loss = adversarial_loss + recognizer_loss
            generator_and_encoder_loss += generator_loss.item()

            # calculate fid
            fid_set.update(fake_image_batch.cpu().repeat(1, 3, 1, 1), real=False)

            # Calculate recognizer error
            preds = create_strings_from_tensor(torch.argmax(recognizer_outputs, dim=1))
            target = create_strings_from_tensor(input_text_batch)
            recognizer_error += calculate_recog_accuracy(preds, target)
            print(f"[EVAL] processed batch {i}, disc_loss={discriminator_loss_eval} | ad_loss={adversarial_loss}, recog_loss={recognizer_loss}")

    generator_and_encoder_error = fid_set.compute().item()
    fid_set.reset()
    discriminator_accuracy /= len(line_loader)
    generator_and_encoder_loss /= len(line_loader)
    discriminator_loss /= len(line_loader)
    recognizer_error /= len(line_loader)

    return generator_and_encoder_loss, discriminator_loss, generator_and_encoder_error, discriminator_accuracy, recognizer_error

"""Training Functions"""
def train_recognizer(recognizer, train_line_loader, val_line_loader, batch_size=64, learning_rate=2e-4, betas=(0, 0.999), num_epochs=30):
    # only train on real images
    # also save model, plot graphs, save graphs
    pass

def train_gan(generator, encoder, discriminator, recognizer,
              train_line_dataset, val_line_dataset,
              batch_size=64, encoder_lr=1e-5, generator_lr=1e-5, discriminator_lr=1e-5,
              betas=(0, 0.999), num_epochs=30, recognizer_downscale_factor=1,
              num_generator_updates_per_discriminator_update=1):
    # Note, the generator and discriminator should be spectrally normalized before training
    # TODO: load dataloader with batch size batch_size

    time_gan_init = f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    generator = generator.to(device)
    encoder = encoder.to(device)
    discriminator = discriminator.to(device)
    recognizer = recognizer.to(device)

    train_line_loader = DataLoader(train_line_dataset, batch_size=batch_size, shuffle=True)
    val_line_loader = DataLoader(val_line_dataset, batch_size=batch_size, shuffle=True)

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=encoder_lr, betas=betas)
    generator_optimizer = optim.Adam(generator.parameters(), lr=generator_lr, betas=betas)
    discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=discriminator_lr, betas=betas)

    adversarial_loss_function = nn.BCELoss()  # discriminator already has sigmoid
    recognizer_loss_function = nn.NLLLoss()
    saving_filenames = {
        "encoder": os.path.join("main_model", "model_snapshots", f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_bs{batch_size}_lrE{encoder_lr}_lrG{generator_lr}_lrD{discriminator_lr}_betas{betas}_encoder"),
        "generator": os.path.join("main_model", "model_snapshots", f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_bs{batch_size}_lrE{encoder_lr}_lrG{generator_lr}_lrD{discriminator_lr}_betas{betas}_generator"),
        "discriminator": os.path.join("main_model", "model_snapshots", f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_bs{batch_size}_lrE{encoder_lr}_lrG{generator_lr}_lrD{discriminator_lr}_betas{betas}_discriminator"),
        "losses": os.path.join("main_model", "model_training_information", f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_bs{batch_size}_lrE{encoder_lr}_lrG{generator_lr}_lrD{discriminator_lr}_betas{betas}_losses"),
        "accuracies": os.path.join("main_model", "model_training_information", f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_bs{batch_size}_lrE{encoder_lr}_lrG{generator_lr}_lrD{discriminator_lr}_accuracies"),
        "raw": os.path.join("main_model", "model_training_information_rawlist", f"{time_gan_init}_bs{batch_size}_lrE{encoder_lr}_lrG{generator_lr}_lrD{discriminator_lr}_rawlist")
        }

    best_generator_and_encoder_val_loss = float('inf')
    best_discriminator_val_loss = float('inf')
    best_generator_and_encoder_val_error = 0
    best_discriminator_val_accuracy = 0
    saved_generator_and_encoder_models_epochs = []
    saved_discriminator_models_epochs = []

    generator_and_encoder_train_losses = []
    discriminator_train_losses = []
    generator_and_encoder_train_accuracies = []
    discriminator_train_accuracies = []
    generator_and_encoder_val_losses = []
    discriminator_val_losses = []
    generator_and_encoder_val_accuracies = []
    discriminator_val_accuracies = []
    recognizer_train_errors = []
    recognizer_val_errors = []

    # FID: https://torchmetrics.readthedocs.io/en/stable/image/frechet_inception_distance.html
    print("FID on train and validation set (all images)")
    fid_set = FrechetInceptionDistance(feature=2048, reset_real_features=False, normalize=True)
    for i, (real_image_batch, _) in enumerate(train_line_loader):
        print(i, real_image_batch.shape)
        fid_set.update(real_image_batch.cpu().repeat(1, 3, 1, 1), real=True)
    for i, (real_image_batch, _) in enumerate(val_line_loader):
        print(i, real_image_batch.shape)
        fid_set.update(real_image_batch.cpu().repeat(1, 3, 1, 1), real=True)


    for epoch in range(num_epochs):
        display_images = []

        generator_and_encoder_train_loss = 0
        discriminator_train_loss = 0
        discriminator_train_accuracy = 0
        recognizer_train_error = 0

        times_discriminator_updated = 0

        for i, (real_image_batch, input_text_batch) in enumerate(train_line_loader):
            print("epoch", epoch, "batch", i)
            # print("real_image_batch.shape", real_image_batch.shape)
            # print("input_text_batch.shape", input_text_batch.shape)

            real_image_batch = real_image_batch.to(device)
            input_text_batch = input_text_batch.to(device)

            # generate noise of N x noise_dim
            noise = torch.randn(len(input_text_batch), generator.noise_dim).to(device)

            # Use encoder and generator to generate fake images
            text_embedding = encoder(input_text_batch)
            # print(noise.shape, text_embedding.shape)
            fake_image_batch = generator(noise, text_embedding)
            # We won't need to pass in real images to recognizer because we use pretrained
            # We pretrained the recognizer ourselves

            # display random image from the batch
            sample_text = input_text_batch[0][input_text_batch[0].nonzero()]
            sample_text = "".join([int_to_char[int(i)] for i in sample_text])
            example = fake_image_batch.cpu().detach().numpy()[0].squeeze(0)

            # Only train discriminator every num_generator_updates_per_discriminator_update iterations
            if i % num_generator_updates_per_discriminator_update == 0:
                times_discriminator_updated += 1
                # train discriminator
                discriminator_optimizer.zero_grad()
                # real_image_batch = real_image_batch.type(torch.float32)
                discriminator_output_for_real_images = discriminator(real_image_batch)
                discriminator_output_for_fake_images = discriminator(fake_image_batch.detach())  # added detatch to prevent gradients from flowing back to generator
                label_for_real_images = torch.ones_like(discriminator_output_for_real_images).to(device)*0.99  # To prevent discriminator from getting too confident
                label_for_fake_images = torch.zeros_like(discriminator_output_for_fake_images).to(device)+0.01  # To prevent discriminator from getting too confident
                real_images_loss = adversarial_loss_function(discriminator_output_for_real_images, label_for_real_images)
                fake_images_loss = adversarial_loss_function(discriminator_output_for_fake_images, label_for_fake_images)
                discriminator_loss = (real_images_loss + fake_images_loss) / 2
                discriminator_loss.backward()  # retain_graph=True because we will use the same discriminator for the generator

                # Gradient clipping to prevent exploding gradients
                torch.nn.utils.clip_grad_norm_(discriminator.parameters(), max_norm=0.5)

                discriminator_optimizer.step()
                discriminator_train_loss += discriminator_loss.item()
                discriminator_train_accuracy += (torch.sum(discriminator_output_for_real_images > 0.5) + torch.sum(discriminator_output_for_fake_images < 0.5)) / (2 * batch_size)
                print("[train] discriminator updated")

            # train generator
            generator_optimizer.zero_grad()
            encoder_optimizer.zero_grad()
            # # generate noise of N x noise_dim
            # noise = torch.randn(len(input_text_batch), generator.noise_dim).to(device)
            # # Use encoder and generator to generate fake images
            # text_embedding = encoder(input_text_batch)
            # fake_image_batch = generator(noise, text_embedding)
            # # display random image from the batch
            # plt.imshow(fake_image_batch.cpu().detach().numpy()[0].squeeze(0), cmap='gray')
            # plt.show()
            discriminator_output_for_fake_images = discriminator(fake_image_batch)
            adversarial_loss = adversarial_loss_function(discriminator_output_for_fake_images, torch.ones_like(discriminator_output_for_fake_images).to(device)*0.99)  # note that we want the fake images to be classified as real, also the *0.99 is to prevent generator from getting too confident
            # print(torch.sum(discriminator_output_for_fake_images > 0.5))
            # print(adversarial_loss)

            # Feed in fake images to the pretrained recognizer for decoding
            recognizer_outputs_from_fake = recognizer(fake_image_batch)
            # Apply NLLLoss()
            recognizer_loss = recognizer_loss_function(
                # Takes input torch.Size([N, 73, 82])
                F.log_softmax(recognizer_outputs_from_fake, 1),
                input_text_batch  # Shape N by 82
            )

            # Calculate recognizer error
            preds = create_strings_from_tensor(torch.argmax(recognizer_outputs_from_fake, dim=1))
            target = create_strings_from_tensor(input_text_batch)
            recognizer_train_error += calculate_recog_accuracy(preds, target)

            # Recognizer loss in general bigger than recognizer+encoder loss
            recognizer_loss = recognizer_loss / recognizer_downscale_factor

            generator_loss = adversarial_loss + recognizer_loss
            generator_loss.backward()
            # We do not update recognizer's gradients here
            # Recognizer does not train if we don't stop its optimizer, gradients just accumulate
            generator_optimizer.step()
            encoder_optimizer.step()
            generator_and_encoder_train_loss += generator_loss.item()

            # calculate FID
            fid_set.update(fake_image_batch.cpu().repeat(1, 3, 1, 1), real=False)

            # display_images.append(fake_image_batch[random.randint(0, len(fake_image_batch) - 1)].detach().numpy())

        # plot the collection of display_images, all are greyscale
        # display_images = np.array(display_images)
        # display_images = np.transpose(display_images, (0, 2, 3, 1))
        # display_images = np.squeeze(display_images)
        # plt.figure(figsize=(10, 10))
        # for i in range(25):
        #     plt.subplot(5, 5, i + 1)
        #     plt.imshow(display_images[i], cmap='gray')
        #     plt.axis('off')
        # plt.tight_layout()
        # plt.show()
        # plt.clf()
        print(sample_text)
        plt.imshow(example, cmap='gray')
        plt.show()

        # Tally up losses and accuracies
        generator_and_encoder_train_loss /= len(train_line_loader)
        discriminator_train_loss /= times_discriminator_updated
        generator_and_encoder_train_error = fid_set.compute().item()
        discriminator_train_accuracy /= times_discriminator_updated
        recognizer_train_error /= len(train_line_loader)
        fid_set.reset()
        # Appending them to the list
        generator_and_encoder_train_losses.append(generator_and_encoder_train_loss)
        discriminator_train_losses.append(discriminator_train_loss)
        generator_and_encoder_train_accuracies.append(generator_and_encoder_train_error)
        discriminator_train_accuracies.append(discriminator_train_accuracy.cpu().numpy())
        recognizer_train_errors.append(recognizer_train_error)

        generator_and_encoder_val_loss, discriminator_val_loss, generator_and_encoder_val_error, discriminator_val_accuracy, recognizer_val_error = calculate_gan_loss_and_accuracies(
            generator,
            encoder,
            discriminator,
            recognizer,
            val_line_loader,
            fid_set, batch_size=batch_size,
            recognizer_downscale_factor=recognizer_downscale_factor
        )
        generator_and_encoder_val_losses.append(generator_and_encoder_val_loss)
        discriminator_val_losses.append(discriminator_val_loss)
        generator_and_encoder_val_accuracies.append(generator_and_encoder_val_error)
        discriminator_val_accuracies.append(discriminator_val_accuracy)
        recognizer_val_errors.append(recognizer_val_error)

        print(f"Epoch {epoch}:\n\tGenerator and encoder train loss: {generator_and_encoder_train_loss}\n\tDiscriminator train loss: {discriminator_train_loss}\n\tGenerator and encoder train error: {generator_and_encoder_train_error}\n\tDiscriminator train accuracy: {discriminator_train_accuracy}\n\tGenerator and encoder val loss: {generator_and_encoder_val_loss}\n\tDiscriminator val loss: {discriminator_val_loss}\n\tGenerator and encoder val error: {generator_and_encoder_val_error}\n\tDiscriminator val accuracy: {discriminator_val_accuracy}\n\tRecognizer train error: {recognizer_train_error}\n\tRecognizer val error: {recognizer_val_error}")

        # Save models, only if they are better than the previous best
        if generator_and_encoder_val_loss < best_generator_and_encoder_val_loss:
            best_generator_and_encoder_val_loss = generator_and_encoder_val_loss
            saved_generator_and_encoder_models_epochs.append(epoch)
            torch.save(generator.state_dict(), f"{saving_filenames['generator']}_epoch{epoch}.pt")
            torch.save(encoder.state_dict(), f"{saving_filenames['encoder']}_epoch{epoch}.pt")
            torch.save(discriminator.state_dict(), f"{saving_filenames['discriminator']}_epoch{epoch}.pt")
        if discriminator_val_loss < best_discriminator_val_loss:
            best_discriminator_val_loss = discriminator_val_loss
            saved_discriminator_models_epochs.append(epoch)
            torch.save(generator.state_dict(), f"{saving_filenames['generator']}_epoch{epoch}.pt")
            torch.save(encoder.state_dict(), f"{saving_filenames['encoder']}_epoch{epoch}.pt")
            torch.save(discriminator.state_dict(), f"{saving_filenames['discriminator']}_epoch{epoch}.pt")
        if generator_and_encoder_val_error < best_generator_and_encoder_val_error:
            best_generator_and_encoder_val_error = generator_and_encoder_val_error
            saved_generator_and_encoder_models_epochs.append(epoch) if epoch not in saved_generator_and_encoder_models_epochs else None
            torch.save(generator.state_dict(), f"{saving_filenames['generator']}_epoch{epoch}.pt")
            torch.save(encoder.state_dict(), f"{saving_filenames['encoder']}_epoch{epoch}.pt")
            torch.save(discriminator.state_dict(), f"{saving_filenames['discriminator']}_epoch{epoch}.pt")
        if discriminator_val_accuracy > best_discriminator_val_accuracy:
            best_discriminator_val_accuracy = discriminator_val_accuracy
            saved_discriminator_models_epochs.append(epoch) if epoch not in saved_discriminator_models_epochs else None
            torch.save(generator.state_dict(), f"{saving_filenames['generator']}_epoch{epoch}.pt")
            torch.save(encoder.state_dict(), f"{saving_filenames['encoder']}_epoch{epoch}.pt")
            torch.save(discriminator.state_dict(), f"{saving_filenames['discriminator']}_epoch{epoch}.pt")

        torch.save(generator.state_dict(), f"{saving_filenames['generator']}_epoch{epoch}_override.pt")
        torch.save(encoder.state_dict(), f"{saving_filenames['encoder']}_epoch{epoch}_override.pt")
        torch.save(discriminator.state_dict(), f"{saving_filenames['discriminator']}_epoch{epoch}_override.pt")
        # Save per epoch
        # save the losses and accuracies as csvs
        with open(f"{saving_filenames['raw']}.txt", "w") as f:
            names = ["generator_and_encoder_train_losses",
                         "discriminator_train_losses",
                         "generator_and_encoder_train_accuracies",
                         "discriminator_train_accuracies",
                         "generator_and_encoder_val_losses",
                         "discriminator_val_losses",
                         "generator_and_encoder_val_accuracies",
                         "discriminator_val_accuracies",
                         "recognizer_train_errors",
                         "recognizer_val_errors",
                         "saved_generator_and_encoder_models_epochs",
                         "saved_discriminator_models_epochs",]

            for i, li in enumerate([generator_and_encoder_train_losses,
                      discriminator_train_losses,
                      generator_and_encoder_train_accuracies,
                      discriminator_train_accuracies,
                      generator_and_encoder_val_losses,
                      discriminator_val_losses,
                      generator_and_encoder_val_accuracies,
                      discriminator_val_accuracies,
                      recognizer_train_errors,
                      recognizer_val_errors,
                      saved_generator_and_encoder_models_epochs,
                      saved_discriminator_models_epochs
            ]):
                f.write(f"{names[i]}: {list(map(float, li))}"+'\n')
        # # plot title
        # plt.title("generator and encoder losses")
        # # plot x axis label
        # plt.xlabel("epoch")
        # # plot y axis label
        # plt.ylabel("loss")
        # # plot the epoch vs the loss
        # plt.plot(range(epoch+1), generator_and_encoder_train_losses, label="train")
        # plt.plot(range(epoch+1), generator_and_encoder_val_losses, label="val")
        # # plot the saved models as dots
        # plt.scatter(saved_generator_and_encoder_models_epochs, [generator_and_encoder_val_losses[i] for i in saved_generator_and_encoder_models_epochs], label="saved models")
        # # legend
        # plt.legend()
        # # show the plot
        # plt.show()
        # # save the plot
        # plt.close()

        # # plot title
        # plt.title("discriminator losses")
        # # plot x axis label
        # plt.xlabel("epoch")
        # # plot y axis label
        # plt.ylabel("loss")
        # # plot the epoch vs the loss
        # plt.plot(range(epoch+1), discriminator_train_losses, label="train")
        # plt.plot(range(epoch+1), discriminator_val_losses, label="val")
        # # plot the saved models as dots
        # plt.scatter(saved_discriminator_models_epochs, [discriminator_val_losses[i] for i in saved_discriminator_models_epochs], label="saved models")
        # # legend
        # plt.legend()
        # # show the plot
        # plt.show()
        # # close the plot
        # plt.close()

        # # plot title
        # plt.title("generator and encoder errors")
        # # plot x axis label
        # plt.xlabel("epoch")
        # # plot y axis label
        # plt.ylabel("accuracy")
        # # plot the epoch vs the accuracy
        # plt.plot(range(epoch+1), generator_and_encoder_train_accuracies, label="train")
        # plt.plot(range(epoch+1), generator_and_encoder_val_accuracies, label="val")
        # # plot the saved models as dots
        # plt.scatter(saved_generator_and_encoder_models_epochs, [generator_and_encoder_val_accuracies[i] for i in saved_generator_and_encoder_models_epochs], label="saved models")
        # # legend
        # plt.legend()
        # # show the plot
        # plt.show()
        # # close the plot
        # plt.close()

        # # plot title
        # plt.title("discriminator accuracies")
        # # plot x axis label
        # plt.xlabel("epoch")
        # # plot y axis label
        # plt.ylabel("accuracy")
        # # plot the epoch vs the accuracy
        # plt.plot(range(epoch+1), discriminator_train_accuracies, label="train")
        # plt.plot(range(epoch+1), discriminator_val_accuracies, label="val")
        # # plot the saved models as dots
        # plt.scatter(saved_discriminator_models_epochs, [discriminator_val_accuracies[i] for i in saved_discriminator_models_epochs], label="saved models")
        # # legend
        # plt.legend()
        # # show the plot
        # plt.show()
        # # close the plot
        # plt.close()

    # Plot the losses and accuracies, and save them (filename should be unique, use datetime as a prefix)
    # The plot should include which epoch's model we saved
    # save the plot, and csv of the losses and accuracies, and which epoch's model we saved

    # plot title
    plt.title("generator and encoder losses")
    # plot x axis label
    plt.xlabel("epoch")
    # plot y axis label
    plt.ylabel("loss")
    # plot the epoch vs the loss
    plt.plot(range(num_epochs), generator_and_encoder_train_losses, label="train")
    plt.plot(range(num_epochs), generator_and_encoder_val_losses, label="val")
    # plot the saved models as dots
    plt.scatter(saved_generator_and_encoder_models_epochs, [generator_and_encoder_val_losses[i] for i in saved_generator_and_encoder_models_epochs], label="saved models")
    # legend
    plt.legend()
    # save the plot
    plt.savefig(f"{saving_filenames['losses']}_generator_and_encoder_loss.png")
    # show the plot
    plt.show()
    # close the plot
    plt.close()

    # plot title
    plt.title("discriminator losses")
    # plot x axis label
    plt.xlabel("epoch")
    # plot y axis label
    plt.ylabel("loss")
    # plot the epoch vs the loss
    plt.plot(range(num_epochs), discriminator_train_losses, label="train")
    plt.plot(range(num_epochs), discriminator_val_losses, label="val")
    # plot the saved models as dots
    plt.scatter(saved_discriminator_models_epochs, [discriminator_val_losses[i] for i in saved_discriminator_models_epochs], label="saved models")
    # legend
    plt.legend()
    # save the plot
    plt.savefig(f"{saving_filenames['losses']}_discriminator_loss.png")
    # show the plot
    plt.show()
    # close the plot
    plt.close()

    # plot title
    plt.title("generator and encoder errors")
    # plot x axis label
    plt.xlabel("epoch")
    # plot y axis label
    plt.ylabel("error")
    # plot the epoch vs the accuracy
    plt.plot(range(num_epochs), generator_and_encoder_train_accuracies, label="train")
    plt.plot(range(num_epochs), generator_and_encoder_val_accuracies, label="val")
    # plot the saved models as dots
    plt.scatter(saved_generator_and_encoder_models_epochs, [generator_and_encoder_val_accuracies[i] for i in saved_generator_and_encoder_models_epochs], label="saved models")
    # legend
    plt.legend()
    # save the plot
    plt.savefig(f"{saving_filenames['accuracies']}_generator_and_encoder_error.png")
    # show the plot
    plt.show()
    # close the plot
    plt.close()

    # plot title
    plt.title("discriminator accuracies")
    # plot x axis label
    plt.xlabel("epoch")
    # plot y axis label
    plt.ylabel("accuracy")
    # plot the epoch vs the accuracy
    plt.plot(range(epoch+1), discriminator_train_accuracies, label="train")
    plt.plot(range(epoch+1), discriminator_val_accuracies, label="val")
    # plot the saved models as dots
    plt.scatter(saved_discriminator_models_epochs, [discriminator_val_accuracies[i] for i in saved_discriminator_models_epochs], label="saved models")
    # legend
    plt.legend()
    # save the plot
    plt.savefig(f"{saving_filenames['accuracies']}_discriminator_accuracy.png")
    # show the plot
    plt.show()
    # close the plot
    plt.close()

    # plot title
    plt.title("recognizer errors")
    # plot x axis label
    plt.xlabel("epoch")
    # plot y axis label
    plt.ylabel("error")
    # plot the epoch vs the accuracy
    plt.plot(range(num_epochs), recognizer_train_errors, label="train")
    plt.plot(range(num_epochs), recognizer_val_errors, label="val")
    # legend
    plt.legend()
    # save the plot
    plt.savefig(f"{saving_filenames['accuracies']}_recognizer_error.png")
    # show the plot
    plt.show()
    # close the plot
    plt.close()


    # save the losses and accuracies as csvs
    with open(f"{saving_filenames['losses']}.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "generator and encoder train loss", "discriminator train loss", "generator and encoder val loss", "discriminator val loss"])
        for i in range(num_epochs):
            writer.writerow([i, generator_and_encoder_train_losses[i], discriminator_train_losses[i], generator_and_encoder_val_losses[i], discriminator_val_losses[i]])
    with open(f"{saving_filenames['accuracies']}.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "generator and encoder train error", "discriminator train accuracy", "recognizer train error", "generator and encoder val error", "discriminator val accuracy", "recognizer val error"])
        for i in range(num_epochs):
            writer.writerow([i, generator_and_encoder_train_accuracies[i], discriminator_train_accuracies[i], recognizer_train_errors[i], generator_and_encoder_val_accuracies[i], discriminator_val_accuracies[i], recognizer_val_errors[i]])


def load_model(model, model_filename):
    """
    Load a model from a file.
    """
    model.load_state_dict(torch.load(model_filename))
    return model


def load_models_of_same_batch(generator, encoder, discriminator, filename_prefix, epoch_number):
    """
    Load the generator, encoder, and discriminator from files.
    """
    generator = load_model(generator, f"{filename_prefix}_generator_epoch{epoch_number}.pt")
    encoder = load_model(encoder, f"{filename_prefix}_encoder_epoch{epoch_number}.pt")
    discriminator = load_model(discriminator, f"{filename_prefix}_discriminator_epoch{epoch_number}.pt")
    return generator, encoder, discriminator

## Define Network Classes

### Generator Network Modules

In [ ]:
class ConditionalBatchNorm2d(nn.Module):
    """
    Input: (N, C, H, W), with condition vector of shape (N, num_conditions)
    Output: (N, C, H, W)

    Conditional Batch Normalization
    Idea obtained from https://arxiv.org/pdf/1809.11096.pdf
    This is a network layer that applies batch normalization to the input tensor, and conditions it on a condition vector
    For the Generator, this allows the network to learn to generate images conditioned on the class label and the noise vector

    This network takes in a condition vector of length num_conditions, and applies batch normalization to the input tensor.
    Then it computes 2 affine parameters (scale and bias) for each channel of the input tensor, conditioned on the condition vector through a linear layer.
    The affine parameters are then applied to the input tensor, and the output is returned.
    """
    def __init__(self, in_channels, num_conditions):
        """
        in_channels: number of channels in the input tensor
        num_conditions: length of the condition vector
        """
        super(ConditionalBatchNorm2d, self).__init__()
        self.in_channels = in_channels

        # batch normalize the input, without using affine parameters
        self.batch_norm = nn.BatchNorm2d(in_channels, affine=False)

        # set up affine parameters conditioned on the condition vector
        self.embed_conditions = nn.Sequential(
            # 512 hidden units are used by https://arxiv.org/pdf/1903.00277.pdf
            nn.Linear(num_conditions, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, in_channels * 2)
        )
        # # https://arxiv.org/pdf/1809.11096.pdf
        # # initialize affine parameters to be all zeros for bias and ones for scale
        # self.embed_conditions[-1].weight.data.zero_()
        # self.embed_conditions[-1].bias.data[:in_channels].zero_()  # bias is the second half of the affine parameters
        # self.embed_conditions[-1].bias.data[in_channels:].fill_(1)


    def forward(self, x, conditions):
        """
        x: input tensor of shape (N, C, H, W)
        conditions: condition vector of shape (N, num_conditions)
        """
        # apply batch normalization, out still has shape (N, C, H, W)
        out = self.batch_norm(x)

        # compute affine parameters
        params = self.embed_conditions(conditions)
        # params has shape (N, 2 * C), we split the channel dimension in half into 2 tensors of shape (N, C)
        scale, bias = params.chunk(2, dim=1)

        # Apply spectral normalization to the scale and bias
        scale = scale.view(-1, self.in_channels, 1, 1)
        bias = bias.view(-1, self.in_channels, 1, 1)

        # apply scale and bias. every channel's values are scaled and biased by the channel's own scale and bias value
        out = scale.view(-1, self.in_channels, 1, 1) * out + bias.view(-1, self.in_channels, 1, 1)

        # out has shape (N, C, H, W)
        return out

class ResBlockUp(nn.Module):
    """
    Input: (N, in_channels, H, W), with condition vector of shape (N, num_conditions)
    Output: (N, out_channels, H * 2, W * 2)

    Residual Block for Upsampling
    Idea obtained from https://arxiv.org/pdf/1903.00277.pdf
    This is a network layer that upsamples the input tensor by a factor of 2, and conditions it on a condition vector
    For the Generator, this allows the network to learn to generate images conditioned on the class label and the noise vector

    This network takes in a condition vector of length num_conditions, and upsamples the input tensor by a factor of 2, accounting for the condition vector.
    """
    def __init__(self, in_channels, out_channels, num_conditions):
        """
        in_channels: number of channels in the input tensor
        out_channels: number of channels in the output tensor
        num_conditions: length of the condition vector

        specifications inspired by https://arxiv.org/pdf/1903.00277.pdf
        """
        super(ResBlockUp, self).__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.relu = nn.ReLU(inplace=True)

        self.batch_norm1 = ConditionalBatchNorm2d(in_channels, num_conditions)
        self.conv1 = spectral_norm(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        self.batch_norm2 = ConditionalBatchNorm2d(out_channels, num_conditions)
        self.conv2 = spectral_norm(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))

        if in_channels != out_channels:
            self.conv1x1 = spectral_norm(nn.Conv2d(in_channels, out_channels, kernel_size=1))
        else:
            self.conv1x1 = None

    def forward(self, x, conditions):
        """
        x: input tensor of shape (N, C, H, W)
        conditions: condition vector of shape (N, num_conditions)
        """
        # upsample the input tensor
        out1 = self.upsample(x)
        # depending on if this res_block_up changes the number of channels, we may need to use a 1x1 convolution to change the number of channels
        if self.conv1x1 is not None:
            out1 = self.conv1x1(out1)
        # second part of the res_block_up
        out2 = self.batch_norm1(x, conditions)
        out2 = self.relu(out2)
        out2 = self.upsample(out2)
        out2 = self.conv1(out2)
        out2 = self.batch_norm2(out2, conditions)
        out2 = self.relu(out2)
        out2 = self.conv2(out2)

        # the output has shape (N, out_channels, 2 * H, 2 * W)

        out = out1 + out2
        return out

class SelfAttention(nn.Module):
    """Self attention Layer

    This code is obtained from https://github.com/heykeetae/Self-Attention-GAN/blob/master/sagan_models.py
    """

    def __init__(self,in_dim):
        super(SelfAttention,self).__init__()
        self.chanel_in = in_dim

        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax = nn.Softmax(dim=-1) #

    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) # B X CX(N)
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) # BX (N) X (N)
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N

        out = torch.bmm(proj_value,attention.permute(0,2,1) )
        out = out.view(m_batchsize,C,width,height)

        out = self.gamma*out + x
        return out

### Discriminator Network Modules

In [ ]:
class ResBlockDown(nn.Module):
    """

    """
    def __init__(self, in_channels, out_channels):
        """
        in_channels: number of channels in the input tensor
        out_channels: number of channels in the output tensor

        specifications inspired by https://arxiv.org/pdf/1903.00277.pdf
        """
        super(ResBlockDown, self).__init__()
        self.leaky_relu = nn.LeakyReLU(0.01, inplace=True)
        self.average_pool = nn.AvgPool2d(kernel_size=2, stride=2)

        self.batch_norm1 = nn.BatchNorm2d(in_channels)
        self.conv1 = spectral_norm(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        self.conv2 = spectral_norm(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))

        if in_channels != out_channels:
            self.conv1x1 = spectral_norm(nn.Conv2d(in_channels, out_channels, kernel_size=1))
        else:
            self.conv1x1 = None

    def forward(self, x):
        """
        x: input tensor of shape (N, C, H, W)
        """
        # upsample the input tensor
        out1 = x
        # depending on if this res_block_down changes the number of channels, we may need to use a 1x1 convolution to change the number of channels
        if self.conv1x1 is not None:
            out1 = self.conv1x1(out1)
        out1 = self.average_pool(out1)

        # second part of the res_block_up
        out2 = self.batch_norm1(x)
        out2 = self.leaky_relu(out2)
        out2 = self.conv1(out2)
        out2 = self.batch_norm2(out2)
        out2 = self.leaky_relu(out2)
        out2 = self.conv2(out2)
        out2 = self.average_pool(out2)

        # the output has shape (N, out_channels, H / 2, W / 2)
        out = out1 + out2
        return out

class ResBlock(nn.Module):
    """

    """
    def __init__(self, in_channels, out_channels):
        """
        in_channels: number of channels in the input tensor
        out_channels: number of channels in the output tensor

        specifications inspired by https://arxiv.org/pdf/1903.00277.pdf
        """
        super(ResBlock, self).__init__()
        self.leaky_relu = nn.LeakyReLU(0.01, inplace=True)

        self.batch_norm1 = nn.BatchNorm2d(in_channels)
        self.conv1 = spectral_norm(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        self.conv2 = spectral_norm(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))

        if in_channels != out_channels:
            self.conv1x1 = spectral_norm(nn.Conv2d(in_channels, out_channels, kernel_size=1))
        else:
            self.conv1x1 = None

    def forward(self, x):
        """
        x: input tensor of shape (N, C, H, W)
        """
        # upsample the input tensor
        out1 = x
        # depending on if this res_block_down changes the number of channels, we may need to use a 1x1 convolution to change the number of channels
        if self.conv1x1 is not None:
            out1 = self.conv1x1(out1)

        # second part of the res_block_up
        out2 = self.batch_norm1(x)
        out2 = self.leaky_relu(out2)
        out2 = self.conv1(out2)
        out2 = self.batch_norm2(out2)
        out2 = self.leaky_relu(out2)
        out2 = self.conv2(out2)

        # the output has shape (N, out_channels, H, W)
        out = out1 + out2
        return out

class GlobalSumPooling(nn.Module):
    def forward(self, x):
        """
        x: input tensor of shape (N, C, H, W)

        returns a tensor of shape (N, C)
        """
        return torch.sum(x, dim=(2, 3))

### Define Main Networks

In [ ]:
class Encoder(nn.Module):
    """
    RNN
    Input with a vector representation of an ascii text
    Output a vector embedding of the text
    Purpose is to produce an embedding of the text that includes the relationship between the characters
    Description of the encoder comes from https://arxiv.org/pdf/1903.00277.pdf.
    Although we may end up modifying the dimensions of the hidden state and the embedding vector,
    to fit our needs of processing longer texts.
    """

    def __init__(self, char_embedding_dim=128, hidden_dim=256, num_layers=6, num_chars=73):
        """
        embedding_dim: dimension of the embedding vector
        hidden_dim: dimension of the hidden state of the LSTM
        num_layers: number of layers in the LSTM
        num_chars: number of characters in the vocabulary
        """
        super(Encoder, self).__init__()

        self.char_embedding_dim = char_embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_chars = num_chars

        # TODO: embedding can be from a pretrained model
        self.embedding = nn.Embedding(num_chars, char_embedding_dim)
        # Using bidirectional LSTM. Batch first so that the input is of shape (N, L, C)
        self.lstm = nn.LSTM(char_embedding_dim, hidden_dim, num_layers, bidirectional=True, batch_first=True)

    def forward(self, x):
        """
        x: input tensor of shape (N, L), where L is the length of the text. each element is an integer representing a character (1 to 72)
        """
        # Technically, L is the MAXIMUM length of the text in this batch (cuz padding)

        # First embed each character
        x = self.embedding(x)  # output should be (N, L, char_embedding_dim)

        # Run the LSTM, we will only use the hidden state, the sequence output is not needed
        out, _ = self.lstm(x)

        # TODO: this can be modified to specify how to reshape our output
        # out = out[:, -1, :]
        out, _ = torch.max(out, dim=1)
        # out, _ = torch.mean(out, dim=1)

        # out now have shape (N, hidden_dim * 2)
        return out

class Generator(nn.Module):
    """
    Transposed CNN
    Input with a vector embedding of the text and a noise vector
    Output a 128 x 2048 grayscale image
    Purpose is to produce an image that is a representation of the text, with the noise vector adding some variation
    """

    def __init__(self, noise_dim=96, embedding_dim=512):
        """
        noise_dim: dimension of the noise vector, should be divisible by 6
        embedding_dim: dimension of the embedding vector (2 * hidden_dim of the encoder)

        specifications inspired by https://arxiv.org/pdf/1903.00277.pdf
        """
        super(Generator, self).__init__()

        # We are likely going to need to use 5 ResBlockUp layers to get the image to the desired size of 128 x 2048
        # We will upscale to this from a 1 x 16 tensor
        # 5 ResBlockUp also mean our noise vector will be split into 6 parts.
        self.noise_dim = noise_dim
        self.noise_chunk_size = noise_dim // 6
        self.embedding_dim = embedding_dim
        self.fc = spectral_norm(nn.Linear(self.noise_chunk_size, 256 * 1 * 16))
        self.res_block_up1 = ResBlockUp(256, 256, self.embedding_dim + self.noise_chunk_size)
        self.res_block_up2 = ResBlockUp(256, 128, self.embedding_dim + self.noise_chunk_size)
        self.res_block_up3 = ResBlockUp(128, 64, self.embedding_dim + self.noise_chunk_size)
        self.self_attention = SelfAttention(64)
        self.res_block_up4 = ResBlockUp(64, 32, self.embedding_dim + self.noise_chunk_size)
        self.res_block_up5 = ResBlockUp(32, 16, self.embedding_dim + self.noise_chunk_size)
        self.batch_norm = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.conv = spectral_norm(nn.Conv2d(16, 1, kernel_size=3, padding=1))
        self.sigmoid = nn.Sigmoid()

    def forward(self, noise, embedding):
        """
        noise: noise vector of shape (N, noise_dim)
        embedding: embedding of the text of shape (N, embedding_dim)
        """

        # split the noise vector into 8 parts
        noise_chunks = torch.split(noise, self.noise_chunk_size, dim=1)

        # first input to the network is the first noise chunk
        x = noise_chunks[0]
        # pass the concatenated tensor through the fully connected layer
        x = self.fc(x)  # output is (N, 256 * 1 * 16)
        # reshape the tensor to have the desired shape
        x = x.view(-1, 256, 1, 16)  # (N, 256, 1, 16)
        # pass the tensor through the ResBlockUp layers
        x = self.res_block_up1(x, torch.cat((embedding, noise_chunks[1]), dim=1))  # (N, 256, 2, 32)
        x = self.res_block_up2(x, torch.cat((embedding, noise_chunks[2]), dim=1))  # (N, 128, 4, 64)
        x = self.res_block_up3(x, torch.cat((embedding, noise_chunks[3]), dim=1))  # (N, 64, 8, 128)
        x = self.self_attention(x)  # (N, 64, 8, 128)
        x = self.res_block_up4(x, torch.cat((embedding, noise_chunks[4]), dim=1))  # (N, 32, 16, 256)
        x = self.res_block_up5(x, torch.cat((embedding, noise_chunks[5]), dim=1))  # (N, 16, 32, 512)
        # pass the tensor through the batch norm layer
        x = self.batch_norm(x)  # (N, 16, 32, 512)
        # pass the tensor through the relu layer
        x = self.relu(x)  # (N, 16, 32, 512)
        # pass the tensor through the convolution layer
        x = self.conv(x)  # (N, 1, 32, 512)
        # pass the tensor through the sigmoid
        x = self.sigmoid(x)  # (N, 1, 32, 512)

        return x

class Discriminator(nn.Module):
    """
    CNN
    Input with a 128 x 2048 grayscale image
    Output a probability that the image is real and not generated
    Purpose is to determine if the image is real or generated, to encourage the generator to produce realistic images
    """
    def __init__(self):
        """
        noise_dim: dimension of the noise vector, should be divisible by 8
        embedding_dim: dimension of the embedding vector

        specifications inspired by https://arxiv.org/pdf/1903.00277.pdf
        """
        super(Discriminator, self).__init__()

        self.res_block_down1 = ResBlockDown(1, 16)
        self.res_block_down2 = ResBlockDown(16, 32)
        self.self_attention = SelfAttention(32)
        self.res_block_down3 = ResBlockDown(32, 64)
        self.res_block_down4 = ResBlockDown(64, 128)
        self.res_block_down5 = ResBlockDown(128, 256)
        self.res_block = ResBlock(256, 256)
        self.global_sum_pooling = GlobalSumPooling()
        self.fc = spectral_norm(nn.Linear(256, 1))
        self.sigmoid = nn.Sigmoid()

    def forward(self, image):
        """
        image: image tensor of shape (N, 1, 128, 2048)
        """

        # pass the tensor through the ResBlockDown layers
        x = self.res_block_down1(image)  # (N, 16, 16, 256)
        x = self.res_block_down2(x)   # (N, 32, 8, 128)
        x = self.self_attention(x)  # (N, 32, 8, 128)
        x = self.res_block_down3(x)  # (N, 64, 4, 64)
        x = self.res_block_down4(x)  # (N, 128, 2, 32)
        x = self.res_block_down5(x)  # (N, 256, 1, 16)
        # pass the tensor through the ResBlock layer
        x = self.res_block(x)  # (N, 256, 1, 16)
        # pass the tensor through the global sum pooling layer
        x = self.global_sum_pooling(x)  # (N, 256)
        # pass the tensor through the fully connected layer
        x = self.fc(x)  # (N, 1)
        # pass the tensor through the sigmoid
        x = self.sigmoid(x)  # (N, 1)

        return x
    pass

class Recognizer(nn.Module):
    """
    RNN:
    Input with a N x 1 x 32 x 512 image
    Output a vector representation of the text size N x 73 x (82*2+1)
    Purpose is to recognize the text from the image, to encourage the generator to produce images that are representations of the text
    """

    def __init__(self):
        super().__init__()
        self.name = "recognizer"
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(num_features=8)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(num_features=16)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(4,2))
        self.bn4 = nn.BatchNorm2d(num_features=64)
        #self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(4,2))
        #xself.bn5 = nn.BatchNorm2d(num_features=128)
        #self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(4,2))
        #self.bn6 = nn.BatchNorm2d(num_features=256)
        self.lstm = nn.LSTM(input_size=64, hidden_size=128, num_layers=4, bidirectional=True, batch_first=True)
        self.dense = nn.Linear(256, 73)
        self.dense2 = nn.Linear(505, 82)

        #self.fc = nn.Linear(10, 82)

        self.lrelu = nn.LeakyReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout2d(p=0.2)

    def forward(self, img):
        img = self.bn1(self.lrelu(self.conv1(img)))
        #print(img.shape)
        img = self.bn2(self.lrelu(self.conv2(img)))
        #print(img.shape)
        img = self.bn3(self.lrelu(self.dropout(self.conv3(img))))
        #print(img.shape)
        img = self.bn4(self.lrelu(self.dropout(self.conv4(img))))
        #print(img.shape)
        #img = self.bn5(self.lrelu(self.dropout(self.conv5(img))))
        #print(img.shape)
        # Collapse
        img, _ = torch.max(img, dim=2)
        #print(img.shape)
        img = img.permute(0, 2, 1)
        #print(img.shape)
        img, _ = self.lstm(img)
        #print(img.shape)
        img = self.lrelu(self.dense(img))
        #print(img.shape)
        img = img.permute(0,2,1)
        img = self.lrelu(self.dense2(img))

        #img = self.fc(img)
        #print(img.shape)
        #print(img.shape)
        return img
        # img = torch.stack()
        # img = self.dense(img)

## Network Training and Evaluation

### Train Actual Network

In [ ]:
generator = Generator()
discriminator = Discriminator()
encoder = Encoder()
recognizer = Recognizer()

# Load in the recognizer
recog_path = "../models_export/08-10-2023-15-54-31_model_recognizer_bs8_lr0.0005_epoch46.pt"
recognizer = load_model(recognizer, recog_path)

# generator = load_model(generator, "main_model/model_snapshots/2023-07-13_18-06-43_bs4_lr0.0002_betas(0, 0.999)_generator_epoch9.pt")
# generator, encoder, discriminator = load_models_of_same_batch(generator, encoder, discriminator, filename_prefix="main_model/model_snapshots/2023-07-13_18-06-43_bs4_lr0.0002_betas(0, 0.999)", epoch_number=9)

train_gan(encoder_lr=2e-4, generator_lr=2e-4, discriminator_lr=1e-5,
          num_epochs=2, batch_size=32,
          encoder=encoder, generator=generator ,discriminator=discriminator,
          train_line_dataset=line_dataset_train, val_line_dataset=line_dataset_val,
          recognizer=recognizer, num_generator_updates_per_discriminator_update=1, recognizer_downscale_factor=1)

# Code for Demonstration of Network

Model used in this commit: https://github.com/nabethg/ScribeSmith/blob/ac4399d754888b1c1adf75e4c727575e9ba88e17/main_model.ipynb

## Preprocessing

### Load in exported models

Export the encoder, generator, and recognizer that performed the best during training

In [ ]:
encoder = Encoder()
encoder = load_model(encoder, "../models_export/2023-08-11_02-07-32_bs32_lrE0.0002_lrG0.0002_lrD1e-05_betas(0, 0.999)_encoder_epoch33_override.pt")
generator = Generator()
generator = load_model(generator, "../models_export/2023-08-11_02-07-32_bs32_lrE0.0002_lrG0.0002_lrD1e-05_betas(0, 0.999)_generator_epoch33_override.pt")

final_eval_val_loader = DataLoader(line_dataset_val, batch_size=500, shuffle=True)
images, texts = next(iter(final_eval_val_loader))

recognizer =Recognizer()
recognizer = load_model(recognizer, "../models_export/08-10-2023-15-54-31_model_recognizer_bs8_lr0.0005_epoch46.pt")

### Quotes and Shakespeare Snippets

Shakespeare quotes here represent unseen data

In [ ]:
quotes = [
        "He will give the devil his due.", "I know it pleaseth neither of us well.", "summer shall not fade...", "The evil that men do",
        "To be, or not to be", "Lifes but a walking shadow, ", "Nothing will come of nothing.", "Cry \"havoc!\"", "it was Greek to me.", "What light through yonder window breaks.", "Romeo, Romeo!", "some have greatness thrust upon them.",
        "methinks", "that we are underlings.", "Brevity is the soul of wit.", "doth suffer a sea-change", "do we not laugh?", "A rose by any name would smell as sweet.", "loan oft loses both itself", "I am a man",
        "sinned against than sinning.", "All that glisters is not gold.", "Whats in a name?", "this a dagger", "Beware the Ides of March.", "Get thee to a nunnery.", "heaven and earth, Horatio", "Reputation, reputation, reputation!",
        "heaven and men and devils,", "yet I'll speak.", "die upon a kiss.", "we shall ne'er be younger."
]
# quotes = ['He went to the kitchen for water and found', 'sent from the Government were " unsatisfac-', 'In this cruel process which was', 'said he had been stopped by', 'When the sailing season was past , he sent', 'by free trade . It is that they should be free', 'never her husband , a fact he accepted as inevitable . " I guess', 'wished to follow the sightseers , there seemed', 'This last point is certainly important because almost', 'part-author with Miss Delaney of the script ,', 'Similar ontogenetic differences are apparent', 'but obtained less flour .', 'desert her friends . HAPPY , happy', 'had responded to no other human being .', 'from the west .', 'even if they do sleep .', 'and done with .', 'In A Taste of Honey Mr. Richardson has', 'that the movement will go on record against', 'the bottom , both for screwing to the chines and', 'them . Last night\'s play in the " Play of the', 'Macmillan yesterday morning he was given', 'There was utter imbecility in his blank face . Presently the', 'was not enough . " A FAIR COP , " the B B C', '" After long and earnest consideration , Her', 'Committee of 100 , the anti-nuclear arms', 'clearly , sections of the movement reaffirmed their', 'system of slots , scrapers and worms .', 'Morfydd Owen . Morfydd , who had until', "' What a frightful event ! ' he wrote . ' I tremble ! What", 'of euphausiids , has also been described', 'me at the gate , four o\'clock tomorrow ? " " Why ? " he']

len(quotes)

### Images Correspnding to Quotes

We also load in my handwriting of these quotes, representing unseen images

In [ ]:
HEAD = "./shakespeare_demo_handwritten/"
quote_img_nparrs = []
for quote_img in sorted(os.listdir(HEAD)):
    quote_img_nparr = cv.imread(os.path.join(HEAD, quote_img), cv.IMREAD_GRAYSCALE)
    quote_img_nparr = torch.tensor(quote_img_nparr).unsqueeze(0)
    quote_img_nparrs.append(quote_img_nparr)
quotes_tensor_img = torch.stack(quote_img_nparrs)
print(quotes_tensor_img.shape)

# fjskdjfdkls = DataLoader(line_dataset_val, batch_size=32, shuffle=True)
# quotes_tensor_img = next(iter(fjskdjfdkls))
# quotes_tensor_img = quotes_tensor_img[0]
# print(quotes_tensor_img.shape)

### Initializing Fréchet Inception Distance (FID)

Initialize weights and biases as a part of FID with entire dataset

In [ ]:
all_images = LineDataset("../data/lines_improved.txt")
all_images_loader = DataLoader(all_images, batch_size=128)
device = torch.device("cuda")

all_fid = FrechetInceptionDistance(feature=2048, reset_real_features=False, normalize=True).to(device)
for i, (real_image_batch, _) in enumerate(all_images_loader):
    print(i, real_image_batch.shape)
    all_fid.update(real_image_batch.repeat(1, 3, 1, 1).to(device), real=True)

## Generator Demonstration

### Get batch of 32 Shakespeare quotes (make sure quotes are within the 82 character length)

In [ ]:
NUM_TEXTS = 32
temp = []
for quote in quotes:
    ctoi = torch.tensor([char_to_int[char] for char in quote])
    padded_ctoi = F.pad(ctoi, pad=(0, 82-len(ctoi)))
    temp.append(padded_ctoi)
pass_to_embedding = torch.stack(temp)
noise = torch.randn(NUM_TEXTS, generator.noise_dim)

### Pass them into the generator

In [ ]:
embedding = encoder(pass_to_embedding)
outputs = generator(noise, embedding)
print(outputs.shape)

### Output 5 random generator images with labels

In [ ]:
random_indices = np.random.choice(range(0,32), (5,))
print(random_indices)
random_images = outputs[torch.tensor(random_indices)]
print(random_images.shape)

for i in range(5):
    plt.figure()
    plt.title(quotes[i])
    plt.imshow(random_images[i].squeeze(0).detach().numpy(), cmap="gray")

### Calculate FID on this batch of 32

In [ ]:
all_fid.update(outputs.repeat(1, 3, 1, 1).to(device), real=False)
print(all_fid.compute())

## Recognizer Demonstration

### Show handwriting samples of the shakespeare quotes

In [ ]:
# Variable called quotes_tensor_img
# Show the first 5
# quotes_tensor_img = torch.load("baseline_model_outputs.pt")
for i in range(5):
    print(quotes[i])
    plt.figure()
    # print(quotes_tensor_img[i].squeeze(0))
    plt.imshow(quotes_tensor_img[i].squeeze(0), cmap="gray")

### Pass them into the recognizer

In [ ]:
print(quotes_tensor_img.type(torch.float32)/255)
quote_outputs = recognizer(quotes_tensor_img.type(torch.float32)/255)
quote_outputs = torch.argmax(quote_outputs, dim=1)
print(quote_outputs)
quote_outputs.shape

### Show 5 random recognizer outputs of the handwriting

CER is not terribly high and there are some character and length similarities between predicted and target

In [ ]:
random_indices = np.random.choice(range(0,32), (5,))
random_indices = [0,1,2,3,4]
# print(random_indices)

for idx in random_indices:
    plt.figure()
    label = quote_outputs[idx]
    pred = "".join([int_to_char[int(val)] for val in label[label.nonzero()]])
    target = quotes[idx]
    cer = calculate_recog_accuracy([pred], [target])

    plt.title(pred)
    plt.xlabel(f"Character Error Rate: {cer:.3f}")
    plt.imshow(quotes_tensor_img[idx].squeeze(0).detach().numpy(), cmap="gray")


### Output Generation for Final Report

In [ ]:
# encoder = Encoder()
# encoder = load_model(encoder, "../models_export/2023-08-11_02-07-32_bs32_lrE0.0002_lrG0.0002_lrD1e-05_betas(0, 0.999)_encoder_epoch33_override.pt")
# generator = Generator()
# generator = load_model(generator, "../models_export/2023-08-11_02-07-32_bs32_lrE0.0002_lrG0.0002_lrD1e-05_betas(0, 0.999)_generator_epoch33_override.pt")

# final_eval_val_loader = DataLoader(line_dataset_val, batch_size=32)
# images_, texts = next(iter(final_eval_val_loader))

# texts = create_strings_from_tensor(texts)
# print(texts)

# recognizer =Recognizer()
# recognizer = load_model(recognizer, "./08-10-23-recognizers/08-10-2023-15-54-31_model_recognizer_bs8_lr0.0005_epoch46.pt")
# output_label = recognizer(images)

# noise = torch.randn(32, generator.noise_dim)

# embedding = encoder(texts)
# outputs = generator(noise, embedding)

# print(outputs.shape)
# texts = create_strings_from_tensor(texts)
# print(texts)
# plt.title(texts[0])
# plt.imshow(outputs[0].detach().numpy().squeeze(0), cmap="gray")

# plt.figure()
# plt.title(texts[1])
# plt.imshow(outputs[1].detach().numpy().squeeze(0), cmap="gray")

# plt.figure()
# plt.title(texts[2])
# plt.imshow(outputs[2].detach().numpy().squeeze(0), cmap="gray")
# # discriminator = Discriminator()
# # encoder = Encoder()
# # # recognizer = Recognizer()
# # generator, encoder, discriminator = load_models_of_same_batch(generator, encoder, discriminator, filename_prefix="main_model/model_snapshots/2023-07-13_21-45-53_bs32_lr1e-05_betas(0, 0.999)", epoch_number=40)
# # with torch.no_grad():
# #     for i in range(10):
# #         noise = torch.randn(1, 128)
# #         a = line_transcription_dataset_train[0]
# #         embedding = encoder(a.unsqueeze(0))
# #         print(embedding.shape)
# #         image = generator(noise, embedding)
# #         plt.imshow(image.squeeze(0).squeeze(0).detach().numpy(), cmap='gray')
# #         plt.savefig(f'test{i}.png')


# test2 = torch.argmax(output_label, dim=1)
# print(test2.shape)
# test2_img1 = test2[0]
# plt.figure()
# cer = calculate_recog_accuracy(["".join([int_to_char[int(i)] for i in test2_img1[test2_img1.nonzero()]])], [texts[0]])
# plt.xlabel(f"Character Error Rate: {cer:.3f}")
# plt.title("".join([int_to_char[int(i)] for i in test2_img1[test2_img1.nonzero()]]))
# plt.imshow(images[0].squeeze(0).detach().numpy(), cmap="gray")

# plt.figure()
# test2_img1 = test2[1]
# cer = calculate_recog_accuracy(["".join([int_to_char[int(i)] for i in test2_img1[test2_img1.nonzero()]])], [texts[1]])
# plt.xlabel(f"Character Error Rate: {cer:.3f}")
# plt.title("".join([int_to_char[int(i)] for i in test2_img1[test2_img1.nonzero()]]))
# plt.imshow(images[1].squeeze(0).detach().numpy(), cmap="gray")

# plt.figure()
# test2_img1 = test2[2]
# cer = calculate_recog_accuracy(["".join([int_to_char[int(i)] for i in test2_img1[test2_img1.nonzero()]])], [texts[2]])
# plt.xlabel(f"Character Error Rate: {cer:.3f}")
# plt.title("".join([int_to_char[int(i)] for i in test2_img1[test2_img1.nonzero()]]))
# plt.imshow(images[2].squeeze(0).detach().numpy(), cmap="gray")

In [ ]:
### CER for whole shakespeare batch

print(calculate_recog_accuracy(create_strings_from_tensor(quote_outputs), quotes))
# quotes

In [ ]:
### FID on validation set
all_fid.reset()
final_eval_val_loader = DataLoader(line_dataset_val, batch_size=32, shuffle=True)
images, texts = next(iter(final_eval_val_loader))
all_fid.update(images.repeat(1, 3, 1, 1).to(device), real=False)
print(all_fid.compute())

In [ ]:
texts

In [ ]:
# FID for baseline model
### FID on validation set
all_fid = all_fid.to(torch.device("cpu"))
all_fid.reset()
images = torch.load("./baseline_model_outputs.pt")
all_fid.update(images.repeat(1, 3, 1, 1), real=False)
print(all_fid.compute())

In [ ]:
# 6.
import pytesseract
labels = ['makes illusory any hopes that he may be in-', 'of the diggings there was a roughly-built', "of ever being cured . '"]
imgs = torch.load("baseline_model_outputs.pt")
print(imgs.shape)

def Recognizer_Py(img):
        to_pil_image = ToPILImage()
        img = to_pil_image(img)

        string_rep = pytesseract.image_to_string(img)
        string_rep = string_rep.strip()

        # print(string_rep, "\n")


        # vec_rep = [char_to_int[i] for i in string_rep]
        # embedding = torch.FloatTensor(vec_rep)

        # return embedding
        return string_rep

# print(Recognizer_Py(np.squeeze(imgs)))

plt.title(labels[0])
plt.imshow(imgs[0].squeeze(0), cmap="gray")

plt.figure()
plt.title(labels[1])
plt.imshow(imgs[1].squeeze(0), cmap="gray")

plt.figure()
plt.title(labels[2])
plt.imshow(imgs[2].squeeze(0), cmap="gray")

# print(Recognizer_Py())
# Second section
plt.figure()
pred = Recognizer_Py(images_[0])
plt.title(pred)
cer = calculate_recog_accuracy(preds=[pred], target=[labels[0]])
plt.xlabel(f"Character Error Rate: {cer:.3f}")
plt.imshow(images_[0].squeeze(0), cmap="gray")

plt.figure()
pred = Recognizer_Py(images_[1])
plt.title(pred)
cer = calculate_recog_accuracy(preds=[pred], target=[labels[1]])
plt.xlabel(f"Character Error Rate: {cer:.3f}")
plt.imshow(images_[1].squeeze(0), cmap="gray")

plt.figure()
pred = Recognizer_Py(images_[2])
plt.title(pred)
cer = calculate_recog_accuracy(preds=[pred], target=[labels[2]])
plt.xlabel(f"Character Error Rate: {cer:.3f}")
plt.imshow(images_[2].squeeze(0), cmap="gray")